In [ ]:
### THIS CELL SETS UP THE GOOGLE COLAB ENVIRONMENT. 
### IF RUNNING THIS NOTEBOOK LOCALLY, IT MAY BE SAFELY DELETED.

#@title Install software

#@markdown Install the software by pressing the _Play_ button on the left.

try:
    import google.colab
    RUNNING_IN_COLAB = True
except ImportError:
    RUNNING_IN_COLAB = False
except Exception as e: 
    err = "Could not figure out if runnning in a colab notebook\n"
    raise Exception(err) from e

if RUNNING_IN_COLAB:
    !pip install dataprob


In [ ]:
%matplotlib inline
import dataprob
import numpy as np

def michaelis_menten(vmax=100,km=30,s0=None): 
    return s0/(s0 + km)*vmax

gen_params = {"vmax":300,
              "km":10}

err = 5
num_points = 20

s0 = np.linspace(0,100,num_points)
y_obs = michaelis_menten(s0=s0,**gen_params) + np.random.normal(0,err,num_points)
y_std = err

non_fit_kwargs = {"s0":s0}

f = dataprob.setup(some_function=michaelis_menten,
                   method="ml",
                   non_fit_kwargs=non_fit_kwargs)
f.fit(y_obs=y_obs,
      y_std=y_std)

fig = dataprob.plot_summary(f)

In [ ]:
fig = dataprob.plot_corner(f)

In [ ]:
f.fit_df

In [ ]:
f.fit_quality

### Compare input and fit parameters

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(1,figsize=(6,6))

x = np.array(list(gen_params.values()))
y = np.array(f.fit_df.loc[gen_params.keys(),"estimate"])
yerr = np.array(f.fit_df.loc[gen_params.keys(),"std"])

combined = list(x)
combined.extend(y)
span = (np.max(combined) - np.min(combined))
extend_by = 0.1*span
offset = 0.03*span
min_value = np.min(combined) - extend_by
max_value = np.max(combined) + extend_by

ax.scatter(x,y,s=20,edgecolor='none',facecolor='red',zorder=1)
ax.errorbar(x=x,y=y,yerr=yerr,lw=0,elinewidth=1,capsize=3,color='black',zorder=2)

for i, k in enumerate(gen_params.keys()):
    ax.text(x=x[i] - offset,y=y[i] + offset,s=k)


ax.plot((min_value,max_value),(min_value,max_value),'--',color='gray',zorder=0)

ax.set_xlim(min_value,max_value)
ax.set_ylim(min_value,max_value)
ax.set_xlabel("input parameter value")
ax.set_ylabel("estimated parameter value")
